**Title**: CVSS prediction\
**Description**: Load all pre-trained models to predict CVSS score\
**Developer**: Teck Lim\
**Create date**: 04/06/2021

# Import packages

In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import math
from google.colab import drive

!pip install transformers

     |████████████████████████████████| 2.2MB 11.2MB/s 
     |████████████████████████████████| 3.3MB 38.4MB/s 
     |████████████████████████████████| 870kB 37.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=41a617cc83865aafc1f56655b8d731949503e3bc9e072897dbf792f10918a03d
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# CVSS Calculator

In [3]:
def round_up(input):
    int_input = round(input * 100000)
    if int_input % 10000 == 0:
        return int_input / 100000.0
    else:
        return (math.floor(int_input / 10000) + 1) / 10.0

def get_av_score(metric):
    if metric == 'network':
        return 0.85
    elif metric == 'adjacent':
        return 0.62
    elif metric == 'local':
        return 0.55
    elif metric == 'physical':
        return 0.20
    else:
        raise ValueError('Invalid metric value')

def get_ac_score(metric):
    if metric == 'low':
        return 0.77
    elif metric == 'high':
        return 0.44
    else:
        raise ValueError('Invalid metric value')

def get_pr_score(metric, s):
    if metric == 'none':
        return 0.85
    elif metric == 'low':
        return 0.68 if s == 'changed' else 0.62
    elif metric == 'high':
        return 0.50 if s == 'changed' else 0.27
    else:
        raise ValueError('Invalid metric value')

def get_ui_score(metric):
    if metric == 'none':
        return 0.85
    elif metric == 'required':
        return 0.62
    else:
        raise ValueError('Invalid metric value')

def get_c_score(metric):
    if metric == 'high':
        return 0.56
    elif metric == 'low':
        return 0.22
    elif metric == 'none':
        return 0
    else:
        raise ValueError('Invalid metric value')

def get_i_score(metric):
    if metric == 'high':
        return 0.56
    elif metric == 'low':
        return 0.22
    elif metric == 'none':
        return 0
    else:
        raise ValueError('Invalid metric value')

def get_a_score(metric):
    if metric == 'high':
        return 0.56
    elif metric == 'low':
        return 0.22
    elif metric == 'none':
        return 0
    else:
        raise ValueError('Invalid metric value')

def calculcate_iss(c, i, a):
    return 1 - (1-get_c_score(c)) * (1-get_i_score(i)) * (1-get_a_score(a))

def calculate_impact(s, c, i, a):
    iss = calculcate_iss(c, i, a)
    if s == 'unchanged':
        return 6.42 * iss
    elif s == 'changed':
        return (7.52 * (iss - 0.029)) - (3.25 * (iss - 0.02)**15)
    else:
        raise ValueError('Invalid metric value')

def calculate_exploitability(av, ac, pr, ui, s):
    return 8.22 * get_av_score(av) * get_ac_score(ac) * get_pr_score(pr, s) * get_ui_score(ui)

def calculate_scores(av, ac, pr, ui, s, c, i, a):
    av = av.lower()
    ac = ac.lower()
    pr = pr.lower()
    ui = ui.lower()
    s = s.lower()
    c = c.lower()
    i = i.lower()
    a = a.lower()

    impact = calculate_impact(s, c, i, a)
    exploitability = calculate_exploitability(av, ac, pr, ui, s)
    if impact <= 0:
        base = 0
    if s == 'unchanged':
        base = min((impact + exploitability), 10)
    elif s == 'changed':
        base = min(1.08 * (impact + exploitability), 10)
    return round_up(base), round(impact, 1), round(exploitability, 1)

In [4]:
calculate_scores('Network', 'High', 'Low', 'Required', 'Unchanged', 'Low', 'Low', 'Low')

(4.6, 3.4, 1.2)

## Validation

In [5]:
file_path = './gdrive/Shareddrives/twlim_ucsd_drive/Data/cve_train.csv'
file_path = './gdrive/Shareddrives/twlim_ucsd_drive/Data/cve_test.csv'
df = pd.read_csv(file_path)

In [6]:
for idx, row in df.iterrows():
    av = row['attack_vector'].lower()
    av = 'adjacent' if av == 'adjacent_network' else av
    ac = row['attack_complexity'].lower()
    pr = row['privileges_required'].lower()
    ui = row['user_interaction'].lower()
    s = row['scope'].lower()
    c = row['confidentiality'].lower()
    i = row['integrity'].lower()
    a = row['availability'].lower()
    base_score = row['base_score']
    exploitability_score = row['exploitability_score']
    impact_score = row['impact_score']

    try:
        cal_base_score, cal_impact_score, cal_exploitability_score = calculate_scores(av, ac, pr, ui, s, c, i, a)
    except Exception as e:
        print('Index: {}, {}'.format(idx, row['cve_id']))
        continue

    if base_score != cal_base_score or exploitability_score != cal_exploitability_score or impact_score != cal_impact_score:
        print('Index: {}, {}'.format(idx, row['cve_id']))
        print('Base score: {}, {}'.format(base_score, cal_base_score))
        print('Exploitability score: {}, {}'.format(exploitability_score, cal_exploitability_score))
        print('Impact score: {}, {}'.format(impact_score, cal_impact_score))
        continue

# Load the pre-trained models

In [7]:
av_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
ac_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
ui_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/UI'
pr_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
s_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
c_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
i_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'
a_output_dir = './gdrive/Shareddrives/twlim_ucsd_drive/Model/PR'

In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [9]:
from transformers import BertForSequenceClassification, BertTokenizer

av_model = BertForSequenceClassification.from_pretrained(av_output_dir, output_hidden_states=True)
av_tokenizer = BertTokenizer.from_pretrained(av_output_dir)
av_model.to(device)

ac_model = BertForSequenceClassification.from_pretrained(ac_output_dir, output_hidden_states=True)
ac_tokenizer = BertTokenizer.from_pretrained(ac_output_dir)
ac_model.to(device)

pr_model = BertForSequenceClassification.from_pretrained(pr_output_dir, output_hidden_states=True)
pr_tokenizer = BertTokenizer.from_pretrained(pr_output_dir)
pr_model.to(device)

ui_model = BertForSequenceClassification.from_pretrained(ui_output_dir, output_hidden_states=True)
ui_tokenizer = BertTokenizer.from_pretrained(ui_output_dir)
ui_model.to(device)

s_model = BertForSequenceClassification.from_pretrained(s_output_dir, output_hidden_states=True)
s_tokenizer = BertTokenizer.from_pretrained(s_output_dir)
s_model.to(device)

c_model = BertForSequenceClassification.from_pretrained(c_output_dir, output_hidden_states=True)
c_tokenizer = BertTokenizer.from_pretrained(c_output_dir)
c_model.to(device)

i_model = BertForSequenceClassification.from_pretrained(i_output_dir, output_hidden_states=True)
i_tokenizer = BertTokenizer.from_pretrained(i_output_dir)
i_model.to(device)

a_model = BertForSequenceClassification.from_pretrained(a_output_dir, output_hidden_states=True)
a_tokenizer = BertTokenizer.from_pretrained(a_output_dir)
a_model.to(device)

print('All models loaded')

All models loaded


In [10]:
import torch
def text_to_embedding(tokenizer, model, max_len, in_text):
    encoded_dict = tokenizer.encode_plus(
                        in_text,                      # Sentence to encode.
                        add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
                        max_length = max_len,         # Pad & truncate all sentences.
                        padding='max_length',
                        # pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True, # Construct attn. masks.
                        return_tensors = 'pt',        # Return pytorch tensors.
                    )
    input_ids = encoded_dict['input_ids']
    attn_mask = encoded_dict['attention_mask']

    model.eval()

    input_ids = input_ids.to(device)
    attn_mask = attn_mask.to(device)

    with torch.no_grad():
        result = model(input_ids=input_ids,
                    token_type_ids=None,
                    attention_mask=attn_mask)

    # print(result.hidden_states[12][0][0])
    layer_i = 12
    batch_i = 0
    token_i = 0

    logits = result.logits
    logits = logits.detach().cpu().numpy()

    vec = result.hidden_states[layer_i][batch_i][token_i]
    vec = vec.detach().cpu().numpy()

    return logits, vec

# Metrics prediction

In [17]:
input_text_1 = 'Sudo before 1.6.6 contains an off-by-one error that can result in a heap-based buffer overflow that may allow ' \
      'local users to gain root privileges via special characters in the -p (prompt) argument, which are not properly expanded.'
input_text_2 = 'Ubiquiti Networks EdgeSwitch version 1.7.3 and prior suffer from an improperly neutralized element in an OS command ' \
      'due to lack of protection on the admin CLI, leading to code execution and privilege escalation greater than administrators themselves ' \
      'are allowed. An attacker with access to an admin account could escape the restricted CLI and execute arbitrary shell instructions.'
input_text_3 = 'A "javascript:" url loaded by a malicious page can obfuscate its location by blanking the URL displayed in the addressbar, ' \
      'allowing for an attacker to spoof an existing page without the malicious page\'s address being displayed correctly. This vulnerability affects Firefox < 52.'
input_text_4 = "stack over flow that caused by user inserting long text in chrome browser address url bar"

input_text = input_text_4
len(input_text)

89

In [24]:
import textwrap

wrapper = textwrap.TextWrapper(initial_indent='  ', subsequent_indent='  ', width=120)
print('Embedding: \n\n', wrapper.fill(input_text))

print('\nPredictions:\n')
logits, vec = text_to_embedding(av_tokenizer, av_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_av = 'Network'
elif np.argmax(logits, axis=1) == 1:
    predicted_av = 'Adjacent'
elif np.argmax(logits, axis=1) == 2:
    predicted_av = 'Local'
else:
    predicted_av = 'Physical'
print('  AV: {}'.format(predicted_av))
# print('\nEmbedding shape:', str(vec.shape))

logits, vec = text_to_embedding(ac_tokenizer, ac_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_ac = 'Low'
else:
    predicted_ac = 'High'
print('  AC: {}'.format(predicted_ac))

logits, vec = text_to_embedding(pr_tokenizer, pr_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_pr = 'None'
elif np.argmax(logits, axis=1) == 1:
    predicted_pr = 'Low'
else:
    predicted_pr = 'High'
print('  PR: {}'.format(predicted_pr))

logits, vec = text_to_embedding(ui_tokenizer, ui_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_ui = 'None'
else:
    predicted_ui = 'Required'
print('  UI: {}'.format(predicted_ui))

logits, vec = text_to_embedding(s_tokenizer, s_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_s = 'Unchanged'
else:
    predicted_s = 'Changed'
print('  S : {}'.format(predicted_s))

logits, vec = text_to_embedding(c_tokenizer, c_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_c = 'None'
elif np.argmax(logits, axis=1) == 1:
    predicted_c = 'Low'
else:
    predicted_c = 'High'
print('  C : {}'.format(predicted_c))

logits, vec = text_to_embedding(i_tokenizer, i_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_i = 'None'
elif np.argmax(logits, axis=1) == 1:
    predicted_i = 'Low'
else:
    predicted_i = 'High'
print('  I : {}'.format(predicted_i))

logits, vec = text_to_embedding(a_tokenizer, a_model, 512, input_text)
if np.argmax(logits, axis=1) == 0:
    predicted_a = 'None'
elif np.argmax(logits, axis=1) == 1:
    predicted_a = 'Low'
else:
    predicted_a = 'High'
print('  A : {}'.format(predicted_a))

b, i, e = calculate_scores(predicted_av, predicted_ac, predicted_pr, predicted_ui, 
                         predicted_s, predicted_c, predicted_i, predicted_a)
print('\nCVSS scores:\n')
print('  Base score: {}'.format(b))
print('  Impact score: {}'.format(i))
print('  Exploitability score: {}'.format(e))

Embedding: 

   stack over flow that caused by user inserting long text in chrome browser address url bar

Predictions:

  AV: Network
  AC: Low
  PR: None
  UI: Required
  S : Unchanged
  C : None
  I : None
  A : None

CVSS scores:

  Base score: 2.9
  Impact score: 0.0
  Exploitability score: 2.8


# Semantic similarity search

## KNN

In [ ]:
!pip install faiss
!pip install faiss-gpu

     |████████████████████████████████| 4.7MB 7.4MB/s 
     |████████████████████████████████| 89.4MB 56kB/s 


In [ ]:
file_path = './gdrive/Shareddrives/twlim_ucsd_drive/Data/cve.json'
with open(file_path, 'r') as fp:
    data = json.load(fp) 
len(data)

150600

In [ ]:
cve_id = list()
text = list()
label = list()
for idx in range(len(data)):
    try:
        if data[idx].get('impact') and data[idx]['impact'].get('baseMetricV3'):
            cve_id.append(data[idx]['cve']['CVE_data_meta']['ID'])
            text.append(' '.join([text['value'] for text in data[idx]['cve']['description']['description_data']]))
            label.append(0 if data[idx]['impact']['baseMetricV3']['cvssV3']['userInteraction'] == 'NONE' else 1)
    except KeyError:
        print(idx)
        break

df = pd.DataFrame({'cve_id': cve_id, 'label': label, 'text': text})

In [ ]:
sentences = df.text.values[:1000]
len(sentences)

1000

In [ ]:
vecs = list()
for input_text in sentences:
  logits, vec = text_to_embedding(ui_tokenizer, ui_model, 512, input_text)
  vecs.append(vec)
vecs = np.array(vecs)

In [ ]:
import faiss
import time

cpu_index = faiss.IndexFlatL2(vecs.shape[1])

n_gpu = 1

print('Number of GPU: {} using {}'.format(faiss.get_num_gpus(), n_gpu))

co = faiss.GpuMultipleClonerOptions()
co.shard = True

gpu_index = faiss.index_cpu_to_all_gpus(cpu_index, co=co, ngpu=n_gpu)

t0 = time.time()

gpu_index.add(vecs)

elapsed = time.time() - t0
print('Building index took {} seconds'.format(elapsed))

Number of GPU: 1 using 1
Building index took 0.0015549659729003906 seconds


In [ ]:
def find_top_3(input_text_vec):
  D, I = gpu_index.search(input_text_vec.reshape(1, 768), k=3)

  print('Top 3 results')

  for i in range(I.shape[1]):
    result_i = I[0, i]
    print(result_i)
    cve_id = df.iloc[result_i].cve_id
    text = df.iloc[result_i].text

    print(wrapper.fill(cve_id))
    print(wrapper.fill('L2 distance: {}'.format(D[0, i])))
    print(wrapper.fill(text))
    print('')

In [ ]:
sentences[0]

'Stack-based buffer overflow in the jpc_tsfb_getbands2 function in jpc_tsfb.c in JasPer before 1.900.30 allows remote attackers to have unspecified impact via a crafted image.'

In [ ]:
input_text_1 = 'Stack-based buffer overflow in the jpc_tsfb_getbands2 function in jpc_tsfb.c in JasPer before 1.900.30 allows ' \
      'remote attackers to have unspecified impact via a crafted image.'
input_text_2 = 'Ubiquiti Networks EdgeSwitch version 1.7.3 and prior suffer from an improperly neutralized element in an OS command ' \
      'due to lack of protection on the admin CLI, leading to code execution and privilege escalation greater than administrators themselves ' \
      'are allowed. An attacker with access to an admin account could escape the restricted CLI and execute arbitrary shell instructions.'
input_text_3 = 'A "javascript:" url loaded by a malicious page can obfuscate its location by blanking the URL displayed in the addressbar, ' \
      'allowing for an attacker to spoof an existing page without the malicious page\'s address being displayed correctly. This vulnerability affects Firefox < 52.'
input_text_4 = "stack over flow that caused by user inserting long text in chrome browser address url bar"
input_text = input_text_4

In [ ]:
logits, vec = text_to_embedding(ui_tokenizer, ui_model, 512, input_text)
find_top_3(vec)


Top 3 results
123
  CVE-2020-26519
  L2 distance: 22.360809326171875
  Artifex MuPDF before 1.18.0 has a heap based buffer over-write when parsing JBIG2 files allowing attackers to cause a
  denial of service.

487
  CVE-2020-35522
  L2 distance: 24.911407470703125
  In LibTIFF, there is a memory malloc failure in tif_pixarlog.c. A crafted TIFF document can lead to an abort,
  resulting in a remote denial of service attack.

823
  CVE-2017-9937
  L2 distance: 25.4649658203125
  In LibTIFF 4.0.8, there is a memory malloc failure in tif_jbig.c. A crafted TIFF document can lead to an abort
  resulting in a remote denial of service attack.

